In [1]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 98.0 MB/s eta 0:00:00


In [2]:
import re
import pandas as pd
import os
import shutil

import gdown

from glob import glob
import pdfplumber


In [3]:
# URL to your Drive folder
folder_url = "https://drive.google.com/drive/folders/1rqP0g6AFk0Uu6gprstKnxXLo6XKXnIUM"

# 1. Download all files from the folder URL
# gdown.download_folder with just the URL downloads files to a temp dir and returns a list of file paths.
downloaded_files = gdown.download_folder(url=folder_url, quiet=False, use_cookies=False)

# 2. Move the downloaded files from their temporary location to the current working directory
# downloaded_files is a list of paths to the downloaded files in a temporary directory.
if downloaded_files:
    # Assuming all files are downloaded to the same parent temporary directory
    # We can get the parent directory from the first downloaded file path
    temp_output_folder = os.path.dirname(downloaded_files[0]) if downloaded_files else None

    if temp_output_folder and os.path.exists(temp_output_folder):
        for filename in os.listdir(temp_output_folder):
            src = os.path.join(temp_output_folder, filename)
            dst = os.path.join(".", filename)
            if os.path.isfile(src):
                shutil.move(src, dst)
        # Clean up the temporary directory after moving files
        try:
            os.rmdir(temp_output_folder)
            print(f"Removed temporary directory: {temp_output_folder}")
        except OSError as e:
            print(f"Could not remove temporary directory {temp_output_folder}: {e}")
    elif not downloaded_files:
        print("No files were downloaded from the folder.")
    else:
        print(f"Temporary download directory not found: {temp_output_folder}")


Retrieving folder contents


Processing file 1LhIry28zo1wcRObq_kCYwelsUceqM2Uj Q1-2023-Analyst-Call-Transcript.pdf
Processing file 1-iwhMcYbw5NJEFGZdZ7EY4M9vs_d6uQg Q1-2024-Analyst-Call-Transcript.pdf
Processing file 1Fa5lzTPot_0osMTbqyqAIEtdqstFUwkd Q1-2025-Analyst-Call-Transcript.pdf
Processing file 1bvHOZGHIRM-FBHyu1Yx6jQlOnS01u1nj Q2-2023-Analyst-Call-Transcript.pdf
Processing file 1KysovEoUwGRc37z3dCP9-V6hvP4LNktG Q2-2024-Analyst-Call-Transcript.pdf
Processing file 1bV4DJIRc9olUkUFbCjI305m-blZORQq3 Q3-2023-Analyst-Call-Transcript.pdf
Processing file 1cAiRKQVynTMF2GuhfyJJKmv1JjqPkHuq Q3-2024-Analyst-Call-Transcript.pdf
Processing file 1kSNAQpX9V-A0AMKhroa7sgL2lQwQDru6 Q4-2023-Analyst-Call-Transcript.pdf
Processing file 1Dv_sZm2pjkzl1g3bgqf4j1cqVP8PBBDe Q4-2024-Analyst-Call-Transcript.pdf


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1LhIry28zo1wcRObq_kCYwelsUceqM2Uj
To: /content/Q&A/Q1-2023-Analyst-Call-Transcript.pdf
100%|██████████| 288k/288k [00:00<00:00, 5.08MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-iwhMcYbw5NJEFGZdZ7EY4M9vs_d6uQg
To: /content/Q&A/Q1-2024-Analyst-Call-Transcript.pdf
100%|██████████| 209k/209k [00:00<00:00, 4.01MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Fa5lzTPot_0osMTbqyqAIEtdqstFUwkd
To: /content/Q&A/Q1-2025-Analyst-Call-Transcript.pdf
100%|██████████| 270k/270k [00:00<00:00, 4.86MB/s]
Downloading...
From: https://drive.google.com/uc?id=1bvHOZGHIRM-FBHyu1Yx6jQlOnS01u1nj
To: /content/Q&A/Q2-2023-Analyst-Call-Transcript.pdf
100%|██████████| 275k/275k [00:00<00:00, 4.67MB/s]
Downloading...
From: https://drive.google.com/uc?id=1KysovEoUwGRc37z3dCP9-V6hvP4LNktG
To: /content/Q&A/Q2-2024-Analyst-Call-Transcript.

Removed temporary directory: /content/Q&A



Download completed


In [4]:
# Constant config
bank_name = "Deutsche Bank"
output_excel_presentations = "presentation_slides_with_titles.xlsx"
output_excel_qa = "qa_speaker_blocks_final.xlsx"

In [5]:
deutsche_bank_speakers = {"Christian Sewing", "Ioana Patriniche", "James von Moltke", "Silke-Nicole Szypa"}

In [6]:
# Step 1: Extract text from PDF, skipping first page
def extract_text(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages):
            if i == 0:
                continue
            page_text = page.extract_text(layout=True)
            if page_text:
                text += page_text + "\n"
    return text

In [7]:
# Step 2: Cut before Q&A
def extract_presentation_section(text):
    split = re.split(r"Questions? & Answers?", text, flags=re.IGNORECASE)
    return split[0].strip() if split else ""

In [8]:
# Step 3: Parse slides with titles and create data frame for presentation
def extract_slides_with_titles(text):
    slide_pattern = re.compile(r"(Slide\s\d+\s[–-]\s.+)")
    slides = []
    parts = slide_pattern.split(text)
    for i in range(1, len(parts), 2):
        title = parts[i].strip()
        body = parts[i + 1].strip() if i + 1 < len(parts) else ""

        # Remove empty lines and page numbers
        lines = body.splitlines()
        cleaned = [line.strip() for line in lines if line.strip() and not re.match(r"^\d{1,3}$", line.strip())]

        # Combine to single string for bullet logic
        combined = " ".join(cleaned)

        # Insert line breaks before each bullet-style phrase: "- Capital..."
        # But ensure we keep the very first `-` if it's already there
        combined = re.sub(r"\s*-\s+(?=[A-Z])", r"\n- ", combined)

        # Remove accidental double bullets
        combined = re.sub(r"^- -", r"-", combined, flags=re.MULTILINE)

        slides.append((title, combined.strip()))

    return slides

In [35]:
# Step 4: Build DataFrame
def build_slide_df(slides, year, quarter, filename):
    return pd.DataFrame([{
        "File": filename,
        "Bank Name": bank_name,
        "Year": year,
        "Quarter": quarter,
        "Slide_title": title,
        "Text": text
    } for title, text in slides])

In [36]:
# Step 5: Extract Year/Quarter from filename
def extract_year_quarter(filename):
    match = re.search(r"(Q[1-4])[-_](\d{4})", filename)
    if match:
        quarter_str, year = match.group(1), int(match.group(2))
        quarter = int(quarter_str[1])  # Convert 'Q1' → 1
        return year, quarter
    return None, None

In [37]:
# Step 6: Isolate Q&A section
def extract_qa_section(text):
    match = re.search(r"(Questions? & Answers?.*)", text, re.IGNORECASE | re.DOTALL)
    return match.group(1) if match else ""

In [12]:
# Step 7: Create the Q&A part
# 7.1. Parse speaker blocks (speaker name + speaker text)
def parse_speaker_blocks(text):
    lines = text.splitlines()
    speaker_blocks = []
    current_speaker = None
    current_text = ""

    speaker_line_pattern = re.compile(r"^\s{2,}([A-Z][a-z]+(?: [A-Z][a-z]+)+)\s{2,}(.+)?$")

    for line in lines:
        line = line.rstrip()
        if not line.strip():
            continue

        match = speaker_line_pattern.match(line)
        if match:
            if current_speaker and current_text:
                speaker_blocks.append((current_speaker.strip(), current_text.strip()))
                current_text = ""
            current_speaker = match.group(1)
            first_line = match.group(2) or ""
            current_text = first_line.strip()
        else:
            current_text += " " + line.strip()

    if current_speaker and current_text:
        speaker_blocks.append((current_speaker.strip(), current_text.strip()))

    return speaker_blocks



In [38]:
############################################
import re

def parse_speaker_blocks(text):
    lines = text.splitlines()
    speaker_blocks = []
    current_speaker = None
    current_text = []

    # This pattern matches lines like "Chris Hallam (Goldman Sachs)"
    speaker_line_pattern = re.compile(r"^([A-Z][a-z]+(?: [A-Z][a-z]+)+(?: \([^)]+\))?)\s+(.*)?$")

    for line in lines:
        line = line.strip()
        if not line:
            continue

        match = speaker_line_pattern.match(line)
        if match:
            # Save the current speaker block before starting a new one
            if current_speaker and current_text:
                speaker_blocks.append((current_speaker, " ".join(current_text).strip()))
            current_speaker = match.group(1)
            first_line = match.group(2) or ""
            current_text = [first_line.strip()]
        else:
            if current_text is not None:
                current_text.append(line)

    # Append the last speaker block
    if current_speaker and current_text:
        speaker_blocks.append((current_speaker, " ".join(current_text).strip()))

    return speaker_blocks


In [44]:
import re

def parse_speaker_blocks(text):
    # Normalize whitespace
    text = re.sub(r"\n+", "\n", text)

    # Define a regex to match speaker labels (e.g., "Christian Sewing", "James von Moltke")
    speaker_pattern = re.compile(r"\b([A-Z][a-z]+(?:\s+[A-Z][a-z]+)+)\b\s*:")

    # Split text on speaker labels
    speaker_blocks = []
    current_speaker = None
    current_text = []

    for line in text.split("\n"):
        line = line.strip()
        if not line:
            continue

        match = speaker_pattern.match(line)
        if match:
            # Store previous block
            if current_speaker and current_text:
                speaker_blocks.append((current_speaker, " ".join(current_text)))
            current_speaker = match.group(1)
            current_text = [line[len(match.group(0)):].strip()]
        else:
            if current_speaker:
                current_text.append(line)

    # Append last speaker block
    if current_speaker and current_text:
        speaker_blocks.append((current_speaker, " ".join(current_text)))

    return speaker_blocks


In [45]:
# 7.2 First pass: collect institution map from known mentions
def build_institution_map(blocks):
    institution_map = {}
    for speaker, text in blocks:
        match = re.search(r"\(([^)]+)\)", text)
        if match:
            institution_map[speaker] = match.group(1)
    return institution_map



In [46]:
def build_speaker_df(blocks, institution_map, year, quarter, filename):
    rows = []
    question_counter = 1

    for speaker, text in blocks:
        match = re.search(r"\(([^)]+)\)", text)
        if match:
            institution = match.group(1)
            text_clean = re.sub(r"\([^)]+\)", "", text).strip()
            institution_map[speaker] = institution
        else:
            institution = institution_map.get(speaker)
            if not institution and speaker in deutsche_bank_speakers:
                institution = "Deutsche Bank"
            text_clean = text.strip()

        has_question = "?" in text_clean
        question_number = question_counter if has_question else None
        if has_question:
            question_counter += 1

        rows.append({
            "File": filename,
            "Bank Name": bank_name,
            "Year": year,
            "Quarter": quarter,
            "Speaker name": speaker,
            "Institution": institution,
            "Question Number": question_number,
            "Text": text_clean,
            "flag_question": int(has_question)
        })

    df = pd.DataFrame(rows)
    df["Question Number"] = df["Question Number"].ffill().astype("Int64")  # fill down
    return df


In [47]:
# === MAIN LOOP ===
all_files = glob("Q*-Analyst-Call-Transcript*.pdf")
all_present_dfs = []

all_qa_dfs = []

for pdf_path in sorted(all_files, reverse=True):  # optional: newest first
    year, quarter = extract_year_quarter(pdf_path)
    if not year:
        print(f"Skipping {pdf_path} — quarter/year not found.")
        continue

    print(f"Processing {pdf_path} ({quarter} {year})")
    try:
        text = extract_text(pdf_path)
        presentation_text = extract_presentation_section(text)
        slides = extract_slides_with_titles(presentation_text)
        df = build_slide_df(slides, year, quarter, os.path.basename(pdf_path))
        all_present_dfs.append(df)

        qa_text = extract_qa_section(text)
        speaker_blocks = parse_speaker_blocks(qa_text)
        institution_map = build_institution_map(speaker_blocks)
        df_qa = build_speaker_df(speaker_blocks, institution_map, year, quarter, os.path.basename(pdf_path))
        all_qa_dfs.append(df_qa)
    except Exception as e:
        print(f"Failed to process {pdf_path}: {e}")

# Combine and export
if all_present_dfs:
    final_df = pd.concat(all_present_dfs, ignore_index=True)
    final_df.to_excel(output_excel_presentations, index=False)
    json_output_presentations = output_excel_presentations.replace(".xlsx", ".json")
    final_df.to_json(json_output_presentations, orient="records", indent=2, force_ascii=False)

    print(f"\n Saved to {output_excel_presentations} with {len(final_df)} slides.")
else:
    print(" No slides extracted.")

if all_qa_dfs:
    final_df_qa = pd.concat(all_qa_dfs, ignore_index=True)
    final_df_qa.to_excel(output_excel_qa, index=False)
    print(f"\n Saved to {output_excel_qa} with {len(final_df_qa)} slides.")
    json_output_qa = output_excel_qa.replace(".xlsx", ".json")
    final_df_qa.to_json(json_output_qa, orient="records", indent=2, force_ascii=False)
else:
    print(" No QA extracted.")



Processing Q4-2024-Analyst-Call-Transcript.pdf (4 2024)
Failed to process Q4-2024-Analyst-Call-Transcript.pdf: 'Question Number'
Processing Q4-2023-Analyst-Call-Transcript.pdf (4 2023)
Failed to process Q4-2023-Analyst-Call-Transcript.pdf: 'Question Number'
Processing Q3-2024-Analyst-Call-Transcript.pdf (3 2024)
Failed to process Q3-2024-Analyst-Call-Transcript.pdf: 'Question Number'
Processing Q3-2023-Analyst-Call-Transcript.pdf (3 2023)
Failed to process Q3-2023-Analyst-Call-Transcript.pdf: 'Question Number'
Processing Q2-2024-Analyst-Call-Transcript.pdf (2 2024)
Failed to process Q2-2024-Analyst-Call-Transcript.pdf: 'Question Number'
Processing Q2-2023-Analyst-Call-Transcript.pdf (2 2023)
Failed to process Q2-2023-Analyst-Call-Transcript.pdf: 'Question Number'
Processing Q1-2025-Analyst-Call-Transcript.pdf (1 2025)
Failed to process Q1-2025-Analyst-Call-Transcript.pdf: 'Question Number'
Processing Q1-2024-Analyst-Call-Transcript.pdf (1 2024)
Failed to process Q1-2024-Analyst-Call-Tr

FINBERT

In [42]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline
import spacy
import re

In [43]:
# Load FinBERT for sentiment analysis
model_name = "yiyanghkust/finbert-tone"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
sentiment_pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

Device set to use cpu


In [18]:
# Load spaCy for topic extraction
import nltk
nltk.download('punkt')
nlp = spacy.load("en_core_web_sm")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [19]:
# Helper: Extract top noun chunks or named entities
def extract_topics(text):
    doc = nlp(text)
    topics = set()

    for chunk in doc.noun_chunks:
        if len(chunk.text) > 3:
            topics.add(chunk.text.strip())

    for ent in doc.ents:
        if ent.label_ in {"ORG", "MONEY", "GPE", "PRODUCT", "EVENT"}:
            topics.add(ent.text.strip())

    return ", ".join(topics)

In [20]:
import re

removal_phrases = ["Q&A", "thank you", "thanks", "good morning", "hello",
        "hi", "good afternoon", "good evening","i appreciate it",
        "thank you very much", "thank you for taking my question",
        "cheers","many thanks","which", "sorry"]

def clean_text(text):
    text = text.replace("\n", " ").strip()

    # Remove predefined phrases (case-insensitive)
    for phrase in removal_phrases:
        text = re.sub(rf"\b{re.escape(phrase)}\b[,.\s]*", "", text, flags=re.IGNORECASE)

    # Remove phrases like "Thank you," "Thanks," "Good morning," at the start
    text = re.sub(r"^(thank you|thanks|good morning|hello|hi)[,.\s]*", "", text, flags=re.IGNORECASE)

    # Remove question-related filler phrases
    text = re.sub(r"\b(the question|first question|second question|two questions|a quick question|just one question)\b[,.\s]*", "", text, flags=re.IGNORECASE)

    # Remove name after greeting (e.g., "James von Moltke,")
    text = re.sub(r"^[A-Z][a-z]+(?:\s+[A-Z][a-z]+)*[,.\s]+", "", text)

    return text.strip()

In [21]:
def add_sentiment_and_topic(df):
    sentiments = []
    topics = []
    scores = []

    for text in df["Text"]:
        cleaned = clean_text(text)
        if len(cleaned.split()) < 10:
            sentiments.append(None)
            topics.append(None)
            scores.append(None)
        else:
            result = sentiment_pipeline(cleaned[:512])[0]
            sentiments.append(result["label"].lower())
            topics.append(extract_topics(cleaned))
            scores.append(round(result["score"], 4))

    # Add to DataFrame
    df["Sentiment"] = sentiments
    df["Score"] = scores
    df["Topics"] = topics
    return df


In [22]:
add_sentiment_and_topic(final_df_qa)

,File,Bank Name,Year,Quarter,Speaker name,Institution,Question Number,Text,flag_question,Sentiment,Score,Topics
0,Q4-2024-Analyst-Call-Transcript.pdf,Deutsche Bank,2024,4,Nicolas Payen,Kepler Cheuvreux,1,"Good morning. I have two questions, please. Th...",1,neutral,0.9994,"the year, a share buyback annual growth, € 750..."
1,Q4-2024-Analyst-Call-Transcript.pdf,Deutsche Bank,2024,4,Christian Sewing,EU,2,"Thank you, Nicolas, it's Christian. Let me let...",1,positive,1.0000,"US, a chance, those years, the discussions, ou..."
2,Q4-2024-Analyst-Call-Transcript.pdf,Deutsche Bank,2024,4,Anke Reingen,RBC,3,Thank you very much for taking my question. I ...,1,neutral,0.9983,"FX, technology, the divisional level, a great ..."
3,Q4-2024-Analyst-Call-Transcript.pdf,Deutsche Bank,2024,4,Christian Sewing,EU,3,"Anke, I 100% support what James is saying. Jus...",0,neutral,0.9887,"our prepared remarks, that, Rebecca, the outco..."
4,Q4-2024-Analyst-Call-Transcript.pdf,Deutsche Bank,2024,4,Kian Abouhoussein,JP Morgan,4,Thanks for taking my questions. I just wanted ...,1,neutral,0.9986,"the expense side, the opportunity, FX, risk-we..."
...,...,...,...,...,...,...,...,...,...,...,...,...
229,Q1-2023-Analyst-Call-Transcript.pdf,Deutsche Bank,2023,1,Andrew Lim,Societe Generale,17,Hi. Thanks for taking my question. I just have...,1,neutral,0.9945,"Group NII, the coming quarters, a 40 to 60 bas..."
230,Q1-2023-Analyst-Call-Transcript.pdf,Deutsche Bank,2023,1,Andrew Lim,Societe Generale,17,"It's the total Group deposit base, but I can c...",0,neutral,1.0000,"that, the total Group deposit base, that ratio..."
231,Q1-2023-Analyst-Call-Transcript.pdf,Deutsche Bank,2023,1,Andrew Lim,Societe Generale,17,"Great, thanks. James von Moltke Thanks, Andrew.",0,None,NaN,None
232,Q1-2023-Analyst-Call-Transcript.pdf,Deutsche Bank,2023,1,Andrew Lim,Societe Generale,17,"Sorry, lastly, on the impacts due to internal ...",0,neutral,0.9950,"LGDs, the Investment Bank, either the Investme..."


In [23]:
add_sentiment_and_topic(final_df)

,File,Bank Name,Year,Quarter,Slide_title,Text,Sentiment,Score,Topics
0,Q4-2024-Analyst-Call-Transcript.pdf,Deutsche Bank,2024,4,Slide 2 – Actions taken in 2024 position Deuts...,"target in 2025 and beyond\n- Thank you Ioana, ...",negative,0.9975,"2 billion euros year, capital returns, a total..."
1,Q4-2024-Analyst-Call-Transcript.pdf,Deutsche Bank,2024,4,Slide 3 – Resilient full-year results reflecti...,performance\n- We increased 2024 pre-provision...,positive,0.9999,"our efficiency program, FX, the Postbank takeo..."
2,Q4-2024-Analyst-Call-Transcript.pdf,Deutsche Bank,2024,4,Slide 4 – Clear traction across divisions set ...,higher profitability\n- At our investor day in...,positive,1.0000,"March, the commercial focus, substantially bet..."
3,Q4-2024-Analyst-Call-Transcript.pdf,Deutsche Bank,2024,4,Slide 5 – Strong execution and positioning und...,"trajectory\n- Since 2021, we have delivered a ...",positive,1.0000,"US, these trends, FX, this substantial growth,..."
4,Q4-2024-Analyst-Call-Transcript.pdf,Deutsche Bank,2024,4,Slide 6 – Significantly lower expenses in 2025...,"execution of efficiency measures\n- In 2025, o...",positive,0.8964,our initially-planned mandatory and strategic ...
...,...,...,...,...,...,...,...,...,...
161,Q1-2023-Analyst-Call-Transcript.pdf,Deutsche Bank,2023,1,Slide 20 – Investment Bank,- Revenues for the first quarter were 19% lowe...,positive,1.0000,"improvements, A slight increase, March - Rates..."
162,Q1-2023-Analyst-Call-Transcript.pdf,Deutsche Bank,2023,1,Slide 21 – Private Bank,- Private Bank revenues were 2.4 billion euros...,positive,1.0000,"Wealth Management, inflation impacts, the sale..."
163,Q1-2023-Analyst-Call-Transcript.pdf,Deutsche Bank,2023,1,Slide 22 – Asset Management,- Let me continue with Asset Management on sli...,negative,0.9996,"115 million euros, Passive, management fees, 1..."
164,Q1-2023-Analyst-Call-Transcript.pdf,Deutsche Bank,2023,1,Slide 23 – Corporate & Other,- A reminder that Corporate & Other now includ...,positive,1.0000,"a significant improvement, timing, bank levies..."


In [24]:
# Define financial metrics with possible aliases
FINANCIAL_METRICS = {
    "revenue": ["revenue", "revenue target"],
    "expense": ["expense"],
    "income": ["income", "net income", "operating income", "earnings"],
    "margin": ["margin"],
    "profit": ["profit", "net profit"],
    "loss": ["loss", "LGD", "loss given default"],
    "ebitda": ["ebitda"],
    "ebit": ["ebit"],
    "share buyback": ["share buyback"],
    "distribution": ["distribution"],
    "capital ratio": ["capital ratio", "tier 1 ratio", "capital", "CET1", "CET2"],
    "expenses": ["expenses", "opex"],
    "capex": ["capex"],
    "ROE": ["ROE", "return on equity", "return on tangible equity"],
    "cost-income ratio": ["cost-income ratio", "cost/income ratio"],
    "dividend": ["dividend"],
    "aum": ["assets under management", "aum"],
    "net interest income": ["NII", "net interest income","interest income"],
    "market share": ["market share"],
    "cash flow": ["cash flow"],
    "net margin": ["net margin"],
    "EPS": ["EPS", "earnings per share"],
    "RWA": ["RWA", "risk-weighted assets", "risk weighted assets"]
}

# Build a reverse mapping from alias to canonical metric name
alias_to_canonical = {}
all_metric_aliases = []
for canonical, aliases in FINANCIAL_METRICS.items():
    for alias in aliases:
        alias_lower = alias.lower()
        alias_to_canonical[alias_lower] = canonical
        all_metric_aliases.append(alias_lower)

In [25]:
def build_metric_pattern(aliases):
    # Create regex pattern that matches any of the aliases as whole words (with spaces accounted for)
    return "|".join(
        [r"\b" + re.sub(r"\s+", r"\\s+", re.escape(alias)) + r"\b" for alias in aliases]
    )


In [26]:
def extract_financial_metrics(text):
    text = text.replace("\n", " ").strip()
    results = []

    metric_pattern = build_metric_pattern(all_metric_aliases)
    value_pattern = r"(€\s?\d+(?:\.\d+)?\s*(?:million|billion|thousand|bn|m|k)?)"
    percent_pattern = r"(\d{1,3}(?:\.\d+)?\s*%)"

    # Pattern 1: currency value followed by metric
    value_first = re.findall(
        rf"{value_pattern}\s+(?:of\s+)?({metric_pattern})",
        text,
        flags=re.IGNORECASE
    )

    # Pattern 2: metric followed by currency value (up to 50 chars away)
    metric_first = re.findall(
        rf"({metric_pattern})[^\.]{{0,50}}?{value_pattern}",
        text,
        flags=re.IGNORECASE
    )

    # Pattern 3: metric followed by percentage (up to 50 chars away)
    percent_matches = re.findall(
        rf"({metric_pattern})[^\.]{{0,50}}?{percent_pattern}",
        text,
        flags=re.IGNORECASE
    )

    # Normalize and collect results
    for value, metric in value_first:
        canonical = alias_to_canonical.get(metric.lower().strip(), metric.lower().strip())
        results.append({'metric': canonical, 'value': value.strip()})

    for metric, value in metric_first:
        canonical = alias_to_canonical.get(metric.lower().strip(), metric.lower().strip())
        results.append({'metric': canonical, 'value': value.strip()})

    for metric, value in percent_matches:
        canonical = alias_to_canonical.get(metric.lower().strip(), metric.lower().strip())
        results.append({'metric': canonical, 'value': value.strip()})

    return results


In [27]:
def extract_to_columns(row):
    metrics = extract_financial_metrics(row["Text"])
    # Return a Series of dictionaries or empty strings
    return pd.Series({f"metric{i+1}": metric for i, metric in enumerate(metrics)})

In [28]:
# Apply extraction and expand into new columns
metrics_df_qa = final_df_qa.apply(extract_to_columns, axis=1)

In [29]:
# Concatenate with the original DataFrame
cols_to_fill_str = metrics_df_qa.columns.tolist()
final_df_qa_expanded = pd.concat([final_df_qa, metrics_df_qa], axis=1)
final_df_qa_expanded[cols_to_fill_str] = final_df_qa_expanded[cols_to_fill_str].fillna("")

In [30]:
final_df_qa_expanded.head()

,File,Bank Name,Year,Quarter,Speaker name,Institution,Question Number,Text,flag_question,Sentiment,Score,Topics,metric1,metric2,metric3
0,Q4-2024-Analyst-Call-Transcript.pdf,Deutsche Bank,2024,4,Nicolas Payen,Kepler Cheuvreux,1,"Good morning. I have two questions, please. Th...",1,neutral,0.9994,"the year, a share buyback annual growth, € 750...","{'metric': 'revenue', 'value': '€ 32 billion'}",,
1,Q4-2024-Analyst-Call-Transcript.pdf,Deutsche Bank,2024,4,Christian Sewing,EU,2,"Thank you, Nicolas, it's Christian. Let me let...",1,positive,1.0000,"US, a chance, those years, the discussions, ou...","{'metric': 'net interest income', 'value': '€ ...","{'metric': 'distribution', 'value': '€ 2.1 bil...",
2,Q4-2024-Analyst-Call-Transcript.pdf,Deutsche Bank,2024,4,Anke Reingen,RBC,3,Thank you very much for taking my question. I ...,1,neutral,0.9983,"FX, technology, the divisional level, a great ...","{'metric': 'capital ratio', 'value': '62.5%'}",,
3,Q4-2024-Analyst-Call-Transcript.pdf,Deutsche Bank,2024,4,Christian Sewing,EU,3,"Anke, I 100% support what James is saying. Jus...",0,neutral,0.9887,"our prepared remarks, that, Rebecca, the outco...",,,
4,Q4-2024-Analyst-Call-Transcript.pdf,Deutsche Bank,2024,4,Kian Abouhoussein,JP Morgan,4,Thanks for taking my questions. I just wanted ...,1,neutral,0.9986,"the expense side, the opportunity, FX, risk-we...","{'metric': 'revenue', 'value': '€ 32 billion'}","{'metric': 'expense', 'value': '€ 20.8 billion'}",


In [31]:
output_excel_qa_sentiment = "qa_speaker_blocks_with_sentiments.xlsx"
final_df_qa_expanded.to_excel(output_excel_qa_sentiment, index=False)

In [32]:
# Apply extraction and expand into new columns for presentation part
metrics_df = final_df.apply(extract_to_columns, axis=1)

# Concatenate with the original DataFrame
cols_to_fill_str = metrics_df.columns.tolist()
final_df_expanded = pd.concat([final_df, metrics_df], axis=1)
final_df_expanded[cols_to_fill_str] = final_df_expanded[cols_to_fill_str].fillna("")

In [33]:
final_df_expanded.head()

,File,Bank Name,Year,Quarter,Slide_title,Text,Sentiment,Score,Topics,metric1,metric2,metric3,metric4,metric5
0,Q4-2024-Analyst-Call-Transcript.pdf,Deutsche Bank,2024,4,Slide 2 – Actions taken in 2024 position Deuts...,"target in 2025 and beyond\n- Thank you Ioana, ...",negative,0.9975,"2 billion euros year, capital returns, a total...","{'metric': 'profit', 'value': '19%'}","{'metric': 'income', 'value': '65%'}","{'metric': 'capital ratio', 'value': '13.8%'}",,
1,Q4-2024-Analyst-Call-Transcript.pdf,Deutsche Bank,2024,4,Slide 3 – Resilient full-year results reflecti...,performance\n- We increased 2024 pre-provision...,positive,0.9999,"our efficiency program, FX, the Postbank takeo...","{'metric': 'profit', 'value': '19%'}","{'metric': 'revenue', 'value': '4%'}","{'metric': 'income', 'value': '13%'}",,
2,Q4-2024-Analyst-Call-Transcript.pdf,Deutsche Bank,2024,4,Slide 4 – Clear traction across divisions set ...,higher profitability\n- At our investor day in...,positive,1.0000,"March, the commercial focus, substantially bet...","{'metric': 'revenue', 'value': '9%'}","{'metric': 'revenue', 'value': '5%'}",,,
3,Q4-2024-Analyst-Call-Transcript.pdf,Deutsche Bank,2024,4,Slide 5 – Strong execution and positioning und...,"trajectory\n- Since 2021, we have delivered a ...",positive,1.0000,"US, these trends, FX, this substantial growth,...","{'metric': 'revenue', 'value': '5.9%'}","{'metric': 'revenue', 'value': '4%'}",,,
4,Q4-2024-Analyst-Call-Transcript.pdf,Deutsche Bank,2024,4,Slide 6 – Significantly lower expenses in 2025...,"execution of efficiency measures\n- In 2025, o...",positive,0.8964,our initially-planned mandatory and strategic ...,"{'metric': 'income', 'value': '65%'}",,,,


In [34]:
output_excel_presentations_sentiment = "presentation_slides_with_sentiment.xlsx"
final_df_expanded.to_excel(output_excel_presentations_sentiment, index=False)